# **GROUP PROJECT**

https://link.springer.com/referenceworkentry/10.1007%2F978-0-387-74759-0_345

In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import geopandas as gpd
import random
import pyproj
from shapely.geometry import Point, LineString
import matplotlib.pyplot as plt
import seaborn as sns
import ortools as ort
import datetime as dt
# pysal submodule imports
from pysal.lib import cg, examples
from pysal.explore import spaghetti as spgh

import geopandas as gpd
from shapely.geometry import Point
from ortools.linear_solver import pywraplp
import copy, sys, warnings
from collections import OrderedDict

import matplotlib as mpl
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
from matplotlib.colors import ListedColormap

try:
    from IPython.display import set_matplotlib_formats
    set_matplotlib_formats('retina')
except ImportError:
    pass

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from ortools.sat.python import cp_model
%load_ext line_profiler

# **1. FUNCTIONS**

In [ ]:
def buildCityDF(nCities,gridX=100,gridY=100):
    xs = np.random.randint(0,gridX,nCities)
    ys = np.random.randint(0,gridY,nCities)
    pts = [Point(xs[i],ys[i]) for i in range(0,nCities)]
    demands = np.random.randint(0,101,nCities)
    facilityOneCosts = np.random.randint(0,101,nCities)
    facilityTwoCosts = np.random.randint(100,201,nCities)
    cityIndices = range(0,nCities)
    cities = gpd.GeoDataFrame({"x":xs,"y":ys,"demand":demands,"geometry":pts,"City Index":cityIndices,"Facility One Cost":facilityOneCosts,"Facility Two Cost":facilityTwoCosts})
    return cities

In [ ]:
def buildDistanceMatrix_Euclidean(cities):
    distances = np.zeros((len(cities),len(cities)))
    for i,row in cities.iterrows():
        for j,row2 in cities.iterrows():
            distances[j,i] = row.geometry.distance(row2.geometry)
    return distances

In [ ]:
def assignFacilitiesAndCluster(cities,M = 100,Rvz=[100000,200000],truckCapacityZ = [40000,80000],alpha=75.0,truckCapacity = 80000,fractionTruckRangeAllowed=0.8,nFacility=1,maxTimeLimit=5*60*1000):
    solver_instance = pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING
#     solver_instance = cp_model.CpModel()
    distances = buildDistanceMatrix_Euclidean(cities)
    cij = distances
    n_customers = cij.shape[0]
    range_customers = range(n_customers)
    n_facilityCandidates = cij.shape[1]
    range_facilityCandidates = range(n_facilityCandidates)
    demands = np.array(cities["demand"])
    demands_sum = demands.sum()
    fiz = facilityCost = cities[["Facility One Cost","Facility Two Cost"]].as_matrix()
    zRange = len(Rvz)
    model = pywraplp.Solver("Capacitated Fixed Charge", solver_instance)
    model.set_time_limit(maxTimeLimit)

    xij = {(i,j): model.IntVar(0,1, 'x[%i,%i]' % (i,j)) for i in range_customers for j in range_facilityCandidates} # which facility is served by which 
    yjz = {(j,z): model.IntVar(0,1, 'y[%i,%i]' % (j,z)) for j in range_facilityCandidates for z in range(zRange)} # charging station location decision/ concentrator
    
    obj = [alpha * (cij[0,j]+cij[i,j]+cij[i,0]) * xij[i,j] + alpha*2*cij[0,j]*yjz[j,z] + fiz[i,z] * yjz[j,z] for i in range_customers for j in range_facilityCandidates for z in range(zRange)]
#     obj = [alpha * (cij[0,j]+cij[i,j]+cij[i,0]) * xij[i,j] + fiz[i,z] * yjz[j,z] for i in range_customers for j in range_facilityCandidates for z in range(zRange)]

    # Service Constraint
    for i in range_customers:
        model.Add(model.Sum([xij[i,j] for j in range_facilityCandidates]) == nFacility)
    
    # Capacity Constraint
    for j in range_facilityCandidates:
        model.Add(model.Sum([demands[i] * xij[i,j] for i in range_facilityCandidates]) <= model.Sum([truckCapacityZ[z]*yjz[j,z] for z in range(zRange)]))
    
    # Assignment Constraint
    for i in range_customers:
        for j in range_facilityCandidates:
            model.Add(xij[i,j] <= model.Sum([yjz[j,z] for z in range(zRange)]))
    
    # Maximum Concentratorrs Constraint
    for j in range_facilityCandidates:
        model.Add(model.Sum([yjz[j,z] for j in range_facilityCandidates for z in range(zRange)]) <= M)
        
    # Total tour length constraint
    for j in range_facilityCandidates:
        model.Add(model.Sum([xij[i,j]*cij[i,j] for i in range_facilityCandidates]) <= model.Sum([yjz[j,z]*Rvz[z]*fractionTruckRangeAllowed for z in range(zRange)]))
        
    # Z limitation constraint
    for j in range_facilityCandidates:
        model.Add(model.Sum([yjz[j,z] for z in range(zRange)]) <= 1)

    model.Minimize(model.Sum(obj))
    status = model.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Optimal Solution Found')

    resultGDF = cities.copy(deep=True)
    for z in range(2):
        resultGDF[f"Status: Facility Type {z}"] = [yjz[j,z].solution_value() for j in range_facilityCandidates]
#     resultGDF["Status: Facility Type 0"] = [yjz[j,0].solution_value() for j in range_facilityCandidates]
#     resultGDF["Status: Facility Type 1"] = [yjz[j,1].solution_value() for j in range_facilityCandidates]
    return model,yjz,xij,resultGDF,status

In [14]:
def assignFacilitiesAndCluster(cities,M = 100,Rvz=[100000,200000],truckCapacityZ = [40000,80000],alpha=75.0,truckCapacity = 80000,fractionTruckRangeAllowed=0.8,nFacility=1,maxTimeLimit=5*60*1000,tourLengthViolationPenalty = 500):
    solver_instance = pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING
#     solver_instance = cp_model.CpModel()
    distances = buildDistanceMatrix_Euclidean(cities)
    cij = distances
    n_customers = cij.shape[0]
    range_customers = range(n_customers)
    n_facilityCandidates = cij.shape[1]
    range_facilityCandidates = range(n_facilityCandidates)
    demands = np.array(cities["demand"])
    demands_sum = demands.sum()
    fiz = facilityCost = cities[["Facility One Cost","Facility Two Cost"]].as_matrix()
    zRange = len(Rvz)
    model = pywraplp.Solver("Capacitated Fixed Charge", solver_instance)
    model.set_time_limit(maxTimeLimit)

#     model.parameters.max_time_in_seconds = 60.0 * 5

    xij = {(i,j): model.IntVar(0,1, 'x[%i,%i]' % (i,j)) for i in range_customers for j in range_facilityCandidates} # which facility is served by which 
    yjz = {(j,z): model.IntVar(0,1, 'y[%i,%i]' % (j,z)) for j in range_facilityCandidates for z in range(zRange)} # charging station location decision/ concentrator
#     obj = [alpha * (cij[0,j]+cij[i,j]+cij[i,0]) * xij[i,j] + alpha*2*cij[0,j]*yjz[j,z] + fiz[i,z] * yjz[j,z] for i in range_customers for j in range_facilityCandidates for z in range(zRange)]
#     tourLengthObj = [tourLengthViolationPenalty * model.Sum([yjz[j,z]*Rvz[z]-xij[i,j]*cij[i,j] for i in range_customers for z in range(zRange)]) for j in range_customers]
#     tourLengthObj = [tourLengthViolationPenalty * yjz[j,0]*Rvz[0]+yjz[j,1]*Rvz[1]-xij[i,j]*cij[i,j] for i in range_customers for j in range_facilityCandidates]
#     obj = obj.extend(tourLengthObj)
    obj = [alpha * (cij[0,j]+cij[i,j]+cij[i,0]) * xij[i,j] + alpha*2*cij[0,j]*yjz[j,z] + fiz[i,z] * yjz[j,z] + tourLengthViolationPenalty * yjz[j,0]*Rvz[0]+yjz[j,1]*Rvz[1]-xij[i,j]*cij[i,j] for i in range_customers for j in range_facilityCandidates for z in range(zRange)]

    # Service Constraint
    for i in range_customers:
        model.Add(model.Sum([xij[i,j] for j in range_facilityCandidates]) == nFacility)
    
    # Capacity Constraint
    for j in range_facilityCandidates:
        model.Add(model.Sum([demands[i] * xij[i,j] for i in range_facilityCandidates]) <= model.Sum([truckCapacityZ[z]*yjz[j,z] for z in range(zRange)]))
    
    # Assignment Constraint
    for i in range_customers:
        for j in range_facilityCandidates:
            model.Add(xij[i,j] <= model.Sum([yjz[j,z] for z in range(zRange)]))
    
    # Maximum Concentratorrs Constraint
    for j in range_facilityCandidates:
        model.Add(model.Sum([yjz[j,z] for j in range_facilityCandidates for z in range(zRange)]) <= M)
        
    # Total tour length constraint
#     for j in range_facilityCandidates:
#         model.Add(model.Sum([xij[i,j]*cij[i,j] for i in range_facilityCandidates]) <= model.Sum([yjz[j,z]*Rvz[z]*fractionTruckRangeAllowed for z in range(zRange)]))
        
    # Z limitation constraint
    for j in range_facilityCandidates:
        model.Add(model.Sum([yjz[j,z] for z in range(zRange)]) <= 1)

    model.Minimize(model.Sum(obj))
    status = model.Solve()

    if status == pywraplp.Solver.OPTIMAL:
        print('Optimal Solution Found')

    resultGDF = cities.copy(deep=True)
    for z in range(2):
        resultGDF[f"Status: Facility Type {z}"] = [yjz[j,z].solution_value() for j in range_facilityCandidates]
#     resultGDF["Status: Facility Type 0"] = [yjz[j,0].solution_value() for j in range_facilityCandidates]
#     resultGDF["Status: Facility Type 1"] = [yjz[j,1].solution_value() for j in range_facilityCandidates]
    return model,yjz,xij,resultGDF,status

In [5]:
def convertTourToPathDF(tour,resultGDF):
#     tour = list(resultGDF[resultGDF["assignment"]==tour]["City Index"])
    tour.append(tour[0]) # Return to end
    startNodes = []
    endNodes = []
    startCoords = []
    endCoords = []
    lines = [] # https://gis.stackexchange.com/questions/95670/how-to-create-a-shapely-linestring-from-two-points
    for i in range(0,len(tour)):
        currentCity = tour[i]
        if i < len(tour)-1:
            nextCity = tour[i+1]
        else:
            nextCity = tour[0]
        startNodes.append(currentCity)
        endNodes.append(nextCity)
        startPoint = resultGDF[resultGDF["City Index"]==currentCity].geometry
        startCoords.append(startPoint)
        endPoint = resultGDF[resultGDF["City Index"]==nextCity].geometry
        endCoords.append(endPoint)
        line = LineString([(startPoint.x,startPoint.y),(endPoint.x,endPoint.y)])
        lines.append(line)
    gdf = gpd.GeoDataFrame({"Start Node":startNodes,"End Node":endNodes,"geometry":lines})
    return gdf

In [6]:
# https://developers.google.com/optimization/routing/tsp
"""Simple travelling salesman problem between cities."""

def create_data_model(dfSubset):
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = buildDistanceMatrix_Euclidean(dfSubset.reset_index())
    data['num_vehicles'] = 1
    data['depot'] = 0
    return data


def print_solution(manager, routing, assignment,dfSubset):
    """Prints assignment on console."""
    optimizedRoute = []
#     print(f"Objective: ")
#     print('Objective: {} meters'.format(assignment.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route for vehicle 0:\n'
    route_distance = 0
    orderedDFSubset = dfSubset.reset_index()
    while not routing.IsEnd(index):
        correctedCityIndex = orderedDFSubset.iloc[manager.IndexToNode(index)][0]
        optimizedRoute.append(correctedCityIndex)
#         print(f"******{correctedCityIndex}")
#         print(f"====>{orderedDFSubset.iloc[manager.IndexToNode(index)]}")
#         plan_output += ' {} ->'.format(manager.IndexToNode(index))
        plan_output += ' {} ->'.format(correctedCityIndex)

        previous_index = index
        index = assignment.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
#     plan_output += ' {}\n'.format(manager.IndexToNode(index))
    correctedCityIndex = orderedDFSubset.iloc[manager.IndexToNode(index)][0]
    plan_output += ' {}\n'.format(correctedCityIndex)
#     print(f"******{correctedCityIndex}")
    optimizedRoute.append(correctedCityIndex)
#     print(f"====>{orderedDFSubset.iloc[manager.IndexToNode(index)]}")
    plan_output += f'Route distance: {route_distance/1000.0} km\n'
    print(plan_output)
    return optimizedRoute,route_distance


def main(dfSubset):
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model(dfSubset)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    assignment = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if assignment:
        route,route_distance = print_solution(manager, routing, assignment,dfSubset)
    return route,route_distance

In [7]:
def solveAndPlot(illinoisCities,M=20,Rvz=[200000,400000],truckCapacityZ = [40000,80000],alpha=0.001,fractionTruckRangeAllowed=0.75,maxTimeLimit=5*60*1000):
    model,yjz,xij,resultGDF,status = assignFacilitiesAndCluster(illinoisCities,M=M,Rvz=Rvz,truckCapacityZ=truckCapacityZ,alpha=alpha,fractionTruckRangeAllowed=fractionTruckRangeAllowed,maxTimeLimit=maxTimeLimit)
    assignments = np.zeros((nCities,nCities))
    for k,v in xij.items():
        i = k[0]
        j = k[1]
        assignments[i,j] = v.solution_value()
    cluster = []
    for cityNodeID in range(nCities):
        for rootNodeID in range(nCities):
            lookup = assignments[cityNodeID,rootNodeID]
            if lookup == 1:
                cluster.append(rootNodeID)
    resultGDF["assignment"] = cluster
    uniqueClusters = list(set(cluster))


    fig = plt.figure(figsize=(20,20))
    ax1 = fig.add_subplot(1,2,1)
    ax2 = fig.add_subplot(1,2,2)
    fig.tight_layout()

    illinois.plot(ax=ax1,color="blue",alpha=0.1)
    illinois.sort_values("ALAND",ascending=False).reset_index()[0:nCities].plot(ax=ax1,color="red",alpha=0.2)
    # illinois.sort_values("ALAND",ascending=False).reset_index()[0:40].centroid.plot(ax=ax1,color="white",marker='*')

    colors = ['#'+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(len(uniqueClusters))]
    totalCost = 0.0
    subtourRoutingCosts = {}
    subtourConstructionCosts = {}
    subtourDistances = {}
    totalConstructionCost = np.sum(resultGDF["Facility One Cost"] * resultGDF["Status: Facility Type 0"] + resultGDF["Facility Two Cost"] * resultGDF["Status: Facility Type 1"])
    for i in range(0,len(uniqueClusters)):
        dfSubset = resultGDF[resultGDF["assignment"]==uniqueClusters[i]]
        tour,tourLength = main(dfSubset)
        if resultGDF.iloc[uniqueClusters[i]]["Status: Facility Type 0"]==1:
            facilityType = 0
            vehicleType = "Standard range"
        else:
            facilityType = 1
            vehicleType = "Extended range"
        subtourRoutingCosts[i] = alpha * tourLength
        subtourDistances[i] = tourLength
        # check if constraint violated
        label = f"{resultGDF.iloc[uniqueClusters[i]].Name}: {tourLength/1000:,.0f} KM, {vehicleType} vehicle"
        if tourLength > Rvz[facilityType]:
            label += " CONSTRAINT VIOLATED"
        # plot the nodes
        dfSubset.plot(ax=ax1,color=colors[i],markersize=dfSubset["demand"]/100,label=label)
        # plot the tour
        ls = "dashed"
        if facilityType == 1:
            ls = "dotted"
        convertTourToPathDF(tour,resultGDF).plot(ax=ax1,colors=colors[i],ls=ls,alpha=0.75)
        # plot the facilities
        dfFacility = resultGDF[resultGDF["City Index"]==uniqueClusters[i]]
        dfFacility.plot(ax=ax1,markersize=1000,marker="o",alpha=.75,facecolors='none', edgecolors=colors[i])
    totalRoutingCost = np.sum(list(subtourRoutingCosts.values()))
    totalRoutingDistance = np.sum(list(subtourDistances.values()))
    f""
    ax1.set_title(f"Tours organized by cluster\nNode sizes proportional to demand\nConstruction Cost: ${totalConstructionCost:,.2f}, Routing Distance: {totalRoutingDistance/1000:,.0f} KM\n$Total Cost: {totalConstructionCost + totalRoutingCost:,.2f}")
    ax1.legend()
    # Plot assignments
    plt.imshow(assignments)
    ax2.set_title("Assignment of citites to subtours")
    ax2.set_xlabel("Subtour Root (Flag/Concentrator) Node")
    ax2.set_ylabel("City")

# **2. SET UP CITIES**

In [8]:
illinoisCitiesPath = "Final Presentation\\tl_2016_17_place\\tl_2016_17_place.shp"
illinoisCitiesPath = "Final Presentation\\illinois_projected.shp"
illinois = gpd.read_file(illinoisCitiesPath)

In [16]:
nCities = 20
facilityOneBaseCost = 100000
facilityOneBaseSTD = 10000
facilityTwoBaseCost = 1000000
facilityTwoBaseSTD = 100000
baseDemand = 10000
baseDemandSTD = 10000
demands = np.random.normal(baseDemand, baseDemandSTD, nCities)
demands = [np.abs(demand) for demand in demands]
facilityOneCosts = np.random.normal(facilityOneBaseCost, facilityOneBaseSTD, nCities)
facilityOneCosts = [np.abs(facilityOneCost) for facilityOneCost in facilityOneCosts]
facilityTwoCosts = np.random.normal(facilityTwoBaseCost, facilityTwoBaseSTD, nCities)
facilityTwoCosts = [np.abs(facilityTwoCost) for facilityTwoCost in facilityTwoCosts]
cityIndices = range(nCities)
xs = []
ys = []
pts = []
polygons = []
cityNames = []
illinoisGeoms = illinois.sort_values("ALAND",ascending=False).reset_index()
for i,row in illinoisGeoms[0:nCities].iterrows():
    pts.append(row["geometry"].centroid)
    polygons.append(row["geometry"])
    xs.append(row["geometry"].centroid.x)
    ys.append(row["geometry"].centroid.y)
    cityNames.append(row["NAME"])


illinoisCities = gpd.GeoDataFrame({"x":xs,"y":ys,"demand":demands,"geometry":pts,"polygon":polygons,"City Index":cityIndices,"Facility One Cost":facilityOneCosts,"Facility Two Cost":facilityTwoCosts,"Name":cityNames})

# **3. SOLVE TSP SUB-TOURS**

In [ ]:
%prun solveAndPlot(illinoisCities,M=20,Rvz=[300000,600000],truckCapacityZ=[40000,80000],alpha=0.001,fractionTruckRangeAllowed=0.75,maxTimeLimit=5*60*1000)